In [23]:
# !pip install torch torchvision torchaudio

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from spectral import open_image  # HDR/RAW 파일 로드
from PIL import Image
import torchvision.transforms as transforms
import os

In [25]:
# 🔹 데이터셋 클래스 (HDR + RAW 입력, 마스크는 이미지 파일)
class HyperspectralDataset(Dataset):
    def __init__(self, data_dir, mask_dir, transform=None, threshold=10):
        self.data_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.hdr')])
        self.mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith('.png')])
        self.data_dir = data_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.threshold = threshold

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        # 1️⃣ 하이퍼스펙트럼 큐브 로드
        hdr_path = os.path.join(self.data_dir, self.data_files[idx])
        img = open_image(hdr_path).load()
        img = np.array(img)  # (Height, Width, Channels)

        # 2️⃣ HWC -> CHW 형태 변환
        img = np.transpose(img, (2, 0, 1))  # (Channels, Height, Width)
        img_tensor = torch.tensor(img, dtype=torch.float32)

        # 3️⃣ GT 마스크 로드 및 이진화
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])
        mask = Image.open(mask_path).convert('L')
        mask = np.array(mask)

        # Threshold 적용 (픽셀값 0~255 → 0/1 변환)
        mask_binary = (mask > self.threshold).astype(np.uint8)
        mask_tensor = torch.tensor(mask_binary, dtype=torch.long)  # (Height, Width)

        if self.transform:
            img_tensor = self.transform(img_tensor)

        return img_tensor, mask_tensor, img  # 원본 초분광 이미지도 반환

# HS-CNN + U-Net 전체 모델
class HSCNN_UNet(nn.Module):
    def __init__(self, in_channels, num_classes=2):
        super(HSCNN_UNet, self).__init__()
        self.encoder = HSCNN_Encoder(in_channels, features=64)
        self.decoder = UNet_Decoder(features=256, num_classes=num_classes)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x  # (Batch, Classes, H, W)

In [26]:
# 🔹 데이터셋 로드 및 분할
data_dir = r"C:\Users\Jaehyuk\Desktop\2024 자체 구축 데이터셋\사과\241205 사과 당도\splited"
mask_dir = r"C:\Users\Jaehyuk\Desktop\2024 자체 구축 데이터셋\사과\241205 사과 당도\splited_label"
dataset = HyperspectralDataset(data_dir, mask_dir)
 
# 데이터셋 분할 (7:2:1)
total_size = len(dataset)
train_size = int(0.7 * total_size)
test_size = int(0.2 * total_size)
val_size = total_size - train_size - test_size
train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

# 🔹 데이터로더 생성
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)  # 1개씩 평가

# 🔹 모델 설정
in_channels = 30  # HDR 채널 수
num_classes = 2
model = HSCNN_UNet(in_channels, num_classes)

# 🔹 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🔹 학습
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

HSCNN_UNet(
  (encoder): HSCNN_Encoder(
    (conv1): Conv3d(30, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (conv3): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (pool): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): UNet_Decoder(
    (upconv1): ConvTranspose3d(256, 128, kernel_size=(1, 2, 2), stride=(1, 2, 2))
    (conv1): Conv3d(128, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (upconv2): ConvTranspose3d(64, 32, kernel_size=(1, 2, 2), stride=(1, 2, 2))
    (conv2): Conv3d(32, 2, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  )
)

In [27]:
for epoch in range(10):  # 10 Epoch 학습
    model.train()
    running_loss = 0.0
    
    for images, masks, _ in train_loader:
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {running_loss / len(train_loader):.4f}")

print("Training Finished!")

IndexError: list index out of range

In [ ]:
# 🔹 검증 데이터 예측 & 시각화
model.eval()

with torch.no_grad():
    for i, (image, mask, original_hyperspectral) in enumerate(val_loader):
        image = image.to(device)
        output = model(image)
        output_probs = F.softmax(output, dim=1)  # 확률로 변환
        prediction = torch.argmax(output_probs, dim=1).cpu().numpy()[0]  # (H, W)

        # 원본 초분광 이미지에서 특정 파장대역을 RGB로 변환
        rgb_image = np.stack([
            original_hyperspectral[0][100],  # Red (100번째 밴드)
            original_hyperspectral[0][150],  # Green (150번째 밴드)
            original_hyperspectral[0][200]   # Blue (200번째 밴드)
        ], axis=-1)

        # Normalize RGB 이미지 (0~1 범위)
        rgb_image = (rgb_image - np.min(rgb_image)) / (np.max(rgb_image) - np.min(rgb_image))

        # 원본 GT 마스크 (0과 1로 변환된 것)
        gt_mask = mask.numpy()[0]

        # 📌 시각화
        fig, axs = plt.subplots(1, 3, figsize=(12, 4))

        axs[0].imshow(rgb_image)  # 원본 RGB 변환 초분
